In [1]:
!pip install pyspark
!apt-get install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=fd2f087bafb3c9ec7d0eff0d15b93573049b7f532bb77016e2b85e40bf2746f4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 120899 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u392-ga-1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u392-ga-1~22.04) ...
Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../openjdk-8-jdk-headless_8u392-ga-1~22.0

##Drive on

In [2]:
import pyspark
import numpy as np
import pandas as pd
import os

from pyspark.context import SparkContext
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, broadcast
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm

In [3]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "gdrive/MyDrive/Final Data/"
PATH = "gdrive/MyDrive/추천시스템/정리/data/"
os.listdir(path)

Mounted at /content/gdrive


['1210 all_pp.csv',
 '1210 CosSim_All.csv',
 '1210 CosSim_Title.csv',
 '1210 CosSim_Genres.csv',
 '1210 CosSim_Tag.csv',
 '1210 CosSim_Story.csv',
 '1210 EucSim_Story.csv',
 '1210 EucSim_Tag.csv',
 '1210 EucSim_Genres.csv',
 '1210 EucSim_Title.csv',
 '1210 EucSim_All.csv',
 'df_pred_euc_tag_sp.csv',
 'df_pred_euc_title_sp.csv',
 'df_pred_euc_all_sp.csv',
 'df_pred_euc_story_sp.csv',
 'df_pred_cos_all_sp.csv',
 'df_pred_euc_genre_sp.csv',
 'df_pred_cos_title_sp.csv',
 'df_pred_cos_genre_sp.csv',
 'df_pred_cos_story_sp.csv',
 'df_pred_cos_tag_sp.csv']

In [ ]:
cosSim_All    = pd.read_csv(path + '1210 CosSim_All.csv')
cosSim_Genres = pd.read_csv(path + '1210 CosSim_Genres.csv')
cosSim_Title  = pd.read_csv(path + '1210 CosSim_Title.csv')
cosSim_Tag    = pd.read_csv(path + '1210 CosSim_Tag.csv')
cosSim_Story  = pd.read_csv(path + '1210 CosSim_Story.csv')

In [4]:
EucSim_All    =  pd.read_csv(path + '1210 EucSim_All.csv')
EucSim_Genres = pd.read_csv(path + '1210 EucSim_Genres.csv')
EucSim_Title  = pd.read_csv(path + '1210 EucSim_Title.csv')
EucSim_Tag    = pd.read_csv(path + '1210 EucSim_Tag.csv')
EucSim_Story  = pd.read_csv(path + '1210 EucSim_Story.csv')

In [4]:
k = pd.read_csv(path + "1210 all_pp.csv")
movie = pd.read_csv(PATH + "movies_refined.csv")
rating = pd.read_csv(PATH + 'ratings_refined.csv')

##Merging and preprocessing files

In [5]:
new_data = pd.merge(movie, k, on='movieId', how='right')

In [6]:
new_df = pd.merge(movie, k, on='movieId', how='left')

In [7]:
new_ratings = pd.merge(rating, k, on='movieId', how='left')

In [8]:
title_list = new_data['title_x'].tolist()

In [9]:
movie_id_list = new_data['movieId'].tolist()

###Set Euclidean matrix index

In [11]:
EucSim_All_id    = EucSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_All.columns, movie_id_list)))
EucSim_Genres_id = EucSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Genres.columns, movie_id_list)))
EucSim_Title_id  = EucSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Title.columns, movie_id_list)))
EucSim_Tag_id    = EucSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Tag.columns, movie_id_list)))
EucSim_Story_id  = EucSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(EucSim_Story.columns, movie_id_list)))

In [12]:
EucSim_All    = EucSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_All.columns, title_list)))
EucSim_Genres = EucSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Genres.columns, title_list)))
EucSim_Title  = EucSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Title.columns, title_list)))
EucSim_Tag    = EucSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Tag.columns, title_list)))
EucSim_Story  = EucSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(EucSim_Story.columns, title_list)))

###Set cosine matrix index

In [12]:
cosSim_All_id    = cosSim_All.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_All.columns, movie_id_list)))
cosSim_Genres_id = cosSim_Genres.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Genres.columns, movie_id_list)))
cosSim_Title_id  = cosSim_Title.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Title.columns, movie_id_list)))
cosSim_Tag_id    = cosSim_Tag.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Tag.columns, movie_id_list)))
cosSim_Story_id  = cosSim_Story.set_index(pd.Index(movie_id_list)).rename(columns=dict(zip(cosSim_Story.columns, movie_id_list)))

In [13]:
cosSim_All    = cosSim_All.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_All.columns, title_list)))
cosSim_Genres = cosSim_Genres.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Genres.columns, title_list)))
cosSim_Title  = cosSim_Title.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Title.columns, title_list)))
cosSim_Tag    = cosSim_Tag.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Tag.columns, title_list)))
cosSim_Story  = cosSim_Story.set_index(pd.Index(title_list)).rename(columns=dict(zip(cosSim_Story.columns, title_list)))

###Rating pre-processing

In [10]:
new_ratings_refined = rating[rating['movieId'].isin(movie_id_list)]

In [11]:
new_ratings_refined = pd.merge(new_ratings_refined, movie, on='movieId', how='left')

In [12]:
new_ratings_refined = new_ratings_refined[['userId', 'movieId', 'rating', 'title']]

In [13]:
new_ratings_refined

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100694,610,166534,4.0,Split
100695,610,168248,5.0,John Wick: Chapter Two
100696,610,168250,5.0,Get Out
100697,610,168252,5.0,Logan


##Modeling

### 1. python

In [17]:
from tqdm.notebook import tqdm
import numpy as np
def modeling(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1


        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        pred_y = np.matmul(sub_sim_mat, watched_movies_y).flatten() / sim_N

        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

### 2. Spark

In [ ]:
from pyspark import SparkContext, SparkConf

def calculate_predictions(sub_sim_mat, watched_movies_y, sim_N):
    # Initialize SparkContext
    conf = SparkConf().setAppName("ColabApp").setMaster("local[*]")\
                      .set('spark.executor.memory', '4g')\
                      .set('spark.driver.memory', '4g')
    sc = SparkContext.getOrCreate(conf=conf)

    # Create RDDs for sub_sim_mat, watched_movies_y, and sim_N
    sub_sim_mat_rdd = sc.parallelize(sub_sim_mat, numSlices=4)
    watched_movies_y_rdd = sc.parallelize(watched_movies_y, numSlices=4)
    sim_N_bc = sc.broadcast(sim_N)

    # Calculate pred_y using Spark transformations and actions
    def calculate_pred_y(sub_sim_mat_row):
        return np.dot(sub_sim_mat_row, watched_movies_y) / sim_N_bc.value

    pred_y_rdd = sub_sim_mat_rdd.map(calculate_pred_y)
    pred_y = pred_y_rdd.collect()

    # Flatten the result if necessary
    pred_y_flat = np.array(pred_y).flatten()

    # Stop SparkContext
    sc.stop()

    return pred_y_flat


In [ ]:
def modeling_spark(similarity_matrix, data):
    df_pred_all = pd.DataFrame()
    titles = title_list
    all_users = sorted(data['userId'].unique())
    n_titles = len(titles)

    for user in tqdm(all_users):
        idx = data[data['userId'] == user].index

        # 유사도
        watched_movies = data.loc[idx, 'movieId'].tolist()
        sub_sim_mat = similarity_matrix.loc[watched_movies]
        sub_sim_mat = sub_sim_mat.T.to_numpy()
        sim_N = np.sum(sub_sim_mat, axis=1) + 1

        # 평점 예측
        watched_movies_y = data.loc[idx, 'rating']
        watched_movies_y = np.array(watched_movies_y.tolist()).reshape(-1, 1)

        # pred_y 계산 (PySpark)
        pred_y_spark = calculate_predictions(sub_sim_mat, watched_movies_y, sim_N)

        # pred_y를 NumPy 배열로 변환하고 계산
        pred_y = np.array(pred_y_spark)
        users_list = [user] * n_titles
        cur_pred = pd.DataFrame(zip(users_list, titles, pred_y),
                                columns=['userId', 'title', 'pred_rating'])

        # 결과 기록
        df_pred_all = pd.concat([df_pred_all, cur_pred], axis=0)

    return df_pred_all

In [ ]:
cosSim_All_id

,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193571,193573,193581,193583,193585,193587,193609
1,1.000000,0.876327,0.716804,0.457971,0.617727,0.508218,0.395936,0.849832,0.447652,0.434804,...,0.323645,0.545659,0.465674,0.377197,0.548296,0.508439,0.266175,0.616943,0.506044,0.865413
2,0.875729,1.000000,0.665633,0.475699,0.569404,0.553243,0.416010,0.889508,0.543152,0.444473,...,0.311305,0.410299,0.386816,0.305902,0.479891,0.448355,0.223821,0.636172,0.454245,0.743381
3,0.718801,0.669587,1.000000,0.630407,0.575003,0.569360,0.525845,0.628621,0.443755,0.446823,...,0.298588,0.500797,0.452085,0.348682,0.422717,0.612429,0.260799,0.675633,0.519121,0.855869
4,0.427382,0.445211,0.571714,1.000000,0.762320,0.806226,0.612316,0.391407,0.698074,0.528154,...,0.483952,0.181812,0.249821,0.554674,0.222749,0.170516,0.466419,0.512938,0.178643,0.526398
5,0.635450,0.591340,0.591828,0.862821,1.000000,0.820115,0.646798,0.573423,0.730685,0.557048,...,0.546175,0.286394,0.252738,0.584853,0.353173,0.272550,0.535068,0.424955,0.236189,0.615567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.538648,0.484746,0.633665,0.217032,0.284066,0.298944,0.353738,0.415349,0.276585,0.415184,...,0.632366,0.844378,0.682152,0.554198,0.685501,1.000000,0.678158,0.596529,0.940878,0.601799
193583,0.274540,0.242644,0.264844,0.475825,0.475974,0.472470,0.416743,0.142612,0.429535,0.465356,...,0.829749,0.566794,0.406734,0.738282,0.488074,0.595529,1.000000,0.307969,0.623458,0.313496
193585,0.552452,0.571494,0.601445,0.496902,0.352995,0.432765,0.224916,0.505252,0.294081,0.265834,...,0.391910,0.537624,0.623860,0.486067,0.477113,0.503947,0.272358,1.000000,0.563699,0.596747
193587,0.468306,0.426794,0.475788,0.178339,0.200559,0.297555,0.183868,0.393537,0.292695,0.316790,...,0.611707,0.827140,0.705058,0.482613,0.709672,0.829547,0.616559,0.577450,1.000000,0.525424


##Rating prediction

###Cosine Similarity

####all

In [14]:
df_pred_cos_all_sp = pd.read_csv(path + 'df_pred_cos_all_sp.csv')

In [15]:
df_pred_cos_all = df_pred_cos_all_sp

In [19]:
df_pred_cos_all = modeling(cosSim_All_id,new_ratings_refined)

NameError: ignored

####Story

In [16]:
df_pred_cos_story_sp = pd.read_csv(path + 'df_pred_cos_story_sp.csv')

In [17]:
df_pred_cos_story = df_pred_cos_story_sp

In [ ]:
df_pred_cos_story=modeling(cosSim_Story_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Genre

In [18]:
df_pred_cos_genre_sp = pd.read_csv(path + 'df_pred_cos_genre_sp.csv')

In [19]:
df_pred_cos_genre = df_pred_cos_genre_sp

In [ ]:
df_pred_cos_genre=modeling(cosSim_Genres_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Title

In [20]:
df_pred_cos_title_sp = pd.read_csv(path + 'df_pred_cos_title_sp.csv')

In [21]:
df_pred_cos_title = df_pred_cos_title_sp

In [ ]:
df_pred_cos_title=modeling(cosSim_Title_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Tag

In [22]:
df_pred_cos_tag_sp = pd.read_csv(path + 'df_pred_cos_tag_sp.csv')

In [23]:
df_pred_cos_tag = df_pred_cos_tag_sp

In [ ]:
df_pred_cos_tag=modeling(cosSim_Tag_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Story * Title * Genre * Tag


###Euclidean Similarity

####All

In [24]:
df_pred_euc_all_sp = pd.read_csv(path + 'df_pred_euc_all_sp.csv')

In [25]:
df_pred_euc_all = df_pred_euc_all_sp

In [ ]:
df_pred_euc_all=modeling(EucSim_All_id,new_ratings_refined)

####Story

In [26]:
df_pred_euc_story_sp = pd.read_csv(path + 'df_pred_euc_story_sp.csv')

In [27]:
df_pred_euc_story = df_pred_euc_story_sp

In [ ]:
df_pred_euc_story=modeling(EucSim_Story_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Genre

In [28]:
df_pred_euc_genre_sp = pd.read_csv(path + 'df_pred_euc_genre_sp.csv')

In [29]:
df_pred_euc_genre = df_pred_euc_genre_sp

In [ ]:
df_pred_euc_genre=modeling(EucSim_Genres_id,new_ratings_refined)

####Title

In [30]:
df_pred_euc_title_sp = pd.read_csv(path + 'df_pred_euc_title_sp.csv')

In [31]:
df_pred_euc_title = df_pred_euc_title_sp

In [ ]:
df_pred_euc_title=modeling(EucSim_Title_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Tag

In [32]:
df_pred_cos_tag_sp = pd.read_csv(path + 'df_pred_euc_tag_sp.csv')

In [33]:
df_pred_euc_tag = df_pred_cos_tag_sp

In [ ]:
df_pred_euc_tag=modeling(EucSim_Tag_id,new_ratings_refined)

  0%|          | 0/610 [00:00<?, ?it/s]

####Story * Title * Genre * Tag

## Evaluation

In [34]:
from sklearn.model_selection import train_test_split

_, test_data = train_test_split(new_ratings_refined[['userId', 'title', 'rating']],
                test_size=20000, random_state=1234, stratify=new_ratings_refined['userId']) # 2만개 샘플링
test_data

,userId,title,rating
48238,313,Star Wars: Episode VI - Return of the Jedi,3.0
36923,249,Push,3.5
36015,246,Arizona Dream,4.0
64042,414,Finding Nemo,4.5
85364,555,Batman Returns,3.0
...,...,...,...
556,5,Pinocchio,5.0
10198,66,Beverly Hills Cop,3.0
53959,356,"Bourne Supremacy, The",4.0
35434,239,28 Days Later,5.0


### MAE, MSE, RMSE

In [35]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
def calculate_errors(test_data):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
    mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
    rmse = np.sqrt(mse)

    calculate_errors = []
    print(f"MAE  : {str(round(mae, 2))}")
    print(f"MSE  : {str(round(mse, 2))}")
    print(f"RMSE : {str(round(rmse, 2))}")

    calculate_errors.append(mae)
    calculate_errors.append(mse)
    calculate_errors.append(rmse)

    return calculate_errors

 ### Precision, Recall

In [36]:
import numpy as np

def get_precision_recall_f1(test_data):
    threshold = 3
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    # True positives and negatives
    trues = 1 * (true_y >= threshold)
    pred_trues = 1 * (pred_y >= threshold)

    # Calculate True Positives
    true_positive = sum(trues & pred_trues)

    # Precision Calculation
    predicted_positives = sum(pred_trues)
    precision = true_positive / predicted_positives if predicted_positives else 0

    # Recall Calculation
    false_negatives = sum(trues & ~pred_trues)
    recall = true_positive / (true_positive + false_negatives) if (true_positive + false_negatives) else 0

    # F1-Score Calculation
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

    print(f"Precision: {round(precision, 2)}")
    print(f"Recall: {round(recall, 2)}")
    print(f"F1-Score: {round(f1_score, 2)}")

    result = []
    result.append(precision)
    result.append(recall)
    result.append(f1_score)

    return result


### NDCG

In [38]:
def dcg_at_k(relevances, k=10):
    relevances = np.asfarray(relevances)[:k]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)

    return np.sum(relevances / discounts)

def ndcg_at_k(test_data, k=10):
    true_y = np.array(test_data['rating'])
    pred_y = np.array(test_data['pred_rating'])

    idcg = dcg_at_k(sorted(true_y, reverse=True), k)
    dcg = dcg_at_k(pred_y, k)
    ndcg = dcg / idcg if idcg > 0 else 0

    # 결과 출력
    print(f"NDCG Score : {round(ndcg, 2)}")

    return ndcg

#### Cosine Similarity

#####All

In [39]:
test_data_cos_all = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [40]:
print("Cosine_simility All \n")
cos_all = calculate_errors(test_data_cos_all)
cos_all.append(get_precision_recall_f1(test_data_cos_all))
cos_all.append(ndcg_at_k(test_data_cos_all))

Cosine_simility All 

MAE  : 0.92
MSE  : 1.28
RMSE : 1.13
Precision: 0.86
Recall: 0.59
F1-Score: 0.7
NDCG Score : 0.64


##### Story

In [41]:
test_data_cos_story = pd.merge(test_data, df_pred_cos_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_story

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.350053
1,249,Push,3.5,3.993836
2,246,Arizona Dream,4.0,4.238438
3,414,Finding Nemo,4.5,3.259210
4,555,Batman Returns,3.0,3.517714
...,...,...,...,...
21471,5,Pinocchio,5.0,4.099532
21472,66,Beverly Hills Cop,3.0,4.224719
21473,356,"Bourne Supremacy, The",4.0,4.851292
21474,239,28 Days Later,5.0,3.853555


In [42]:
print("Cosine_simility Story \n")
cos_story = calculate_errors(test_data_cos_story)
cos_story.append(get_precision_recall_f1(test_data_cos_story))
cos_story.append(ndcg_at_k(test_data_cos_story))

Cosine_simility Story 

MAE  : 0.86
MSE  : 1.34
RMSE : 1.16
Precision: 0.85
Recall: 0.92
F1-Score: 0.88
NDCG Score : 0.75


##### Genre

In [43]:
test_data_cos_genre = pd.merge(test_data, df_pred_cos_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_genre

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,2.019610
1,249,Push,3.5,3.863685
2,246,Arizona Dream,4.0,3.182846
3,414,Finding Nemo,4.5,3.272621
4,555,Batman Returns,3.0,5.816681
...,...,...,...,...
21471,5,Pinocchio,5.0,3.592705
21472,66,Beverly Hills Cop,3.0,2.091346
21473,356,"Bourne Supremacy, The",4.0,5.911463
21474,239,28 Days Later,5.0,3.564635


In [44]:
print("Cosine_simility Genre \n")
cos_genre = calculate_errors(test_data_cos_genre)
cos_genre.append(get_precision_recall_f1(test_data_cos_genre))
cos_genre.append(ndcg_at_k(test_data_cos_genre))

Cosine_simility Genre 

MAE  : 1.95
MSE  : 50.93
RMSE : 7.14
Precision: 0.83
Recall: 0.44
F1-Score: 0.57
NDCG Score : 0.66


##### Title

In [45]:
test_data_cos_title= pd.merge(test_data, df_pred_cos_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_title

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.395401
1,249,Push,3.5,3.735772
2,246,Arizona Dream,4.0,4.253825
3,414,Finding Nemo,4.5,3.466393
4,555,Batman Returns,3.0,3.513461
...,...,...,...,...
21471,5,Pinocchio,5.0,3.987160
21472,66,Beverly Hills Cop,3.0,4.054229
21473,356,"Bourne Supremacy, The",4.0,4.680771
21474,239,28 Days Later,5.0,4.027721


In [46]:
print("Cosine_simility Title \n")
cos_title = calculate_errors(test_data_cos_title)
cos_title = get_precision_recall_f1(test_data_cos_title)
cos_title.append(ndcg_at_k(test_data_cos_title))

Cosine_simility Title 

MAE  : 1.33
MSE  : 1726.66
RMSE : 41.55
Precision: 0.84
Recall: 0.94
F1-Score: 0.89
NDCG Score : 0.76


##### Tag

In [47]:
test_data_cos_tag = pd.merge(test_data, df_pred_cos_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_tag

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.674569
1,249,Push,3.5,4.772312
2,246,Arizona Dream,4.0,3.605217
3,414,Finding Nemo,4.5,3.057928
4,555,Batman Returns,3.0,3.532794
...,...,...,...,...
21471,5,Pinocchio,5.0,4.400898
21472,66,Beverly Hills Cop,3.0,3.693597
21473,356,"Bourne Supremacy, The",4.0,3.146278
21474,239,28 Days Later,5.0,3.224949


In [48]:
print("Cosine_simility Tag \n")
cos_tag = calculate_errors(test_data_cos_tag)
cos_tag.append(get_precision_recall_f1(test_data_cos_tag))
cos_tag.append(ndcg_at_k(test_data_cos_tag))

Cosine_simility Tag 

MAE  : 15.5
MSE  : 33496.74
RMSE : 183.02
Precision: 0.86
Recall: 0.74
F1-Score: 0.79
NDCG Score : 0.73


#### Euclidean Similarity

##### All

In [49]:
test_data_euc_all= pd.merge(test_data, df_pred_euc_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_all

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.927208
1,249,Push,3.5,4.830012
2,246,Arizona Dream,4.0,3.911656
3,414,Finding Nemo,4.5,3.774880
4,555,Batman Returns,3.0,3.652823
...,...,...,...,...
21471,5,Pinocchio,5.0,2.256649
21472,66,Beverly Hills Cop,3.0,4.600017
21473,356,"Bourne Supremacy, The",4.0,4.477074
21474,239,28 Days Later,5.0,4.179060


In [50]:
print("Euclidean Similarity All \n")
euc_all = calculate_errors(test_data_euc_all)
euc_all.append(get_precision_recall_f1(test_data_euc_all))
euc_all.append(ndcg_at_k(test_data_euc_all))

Euclidean Similarity All 

MAE  : 0.95
MSE  : 1.47
RMSE : 1.21
Precision: 0.83
Recall: 0.89
F1-Score: 0.86
NDCG Score : 0.82


##### Story

In [51]:
test_data_euc_story = pd.merge(test_data, df_pred_euc_story, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_story

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.997916
1,249,Push,3.5,3.549206
2,246,Arizona Dream,4.0,4.562285
3,414,Finding Nemo,4.5,3.845302
4,555,Batman Returns,3.0,3.300467
...,...,...,...,...
21471,5,Pinocchio,5.0,2.309630
21472,66,Beverly Hills Cop,3.0,3.615569
21473,356,"Bourne Supremacy, The",4.0,3.066924
21474,239,28 Days Later,5.0,4.565764


In [52]:
print("Euclidean Similarity stroy \n")
euc_story = calculate_errors(test_data_euc_story)
euc_story.append(get_precision_recall_f1(test_data_euc_story))
euc_story.append(ndcg_at_k(test_data_euc_story))

Euclidean Similarity stroy 

MAE  : 0.87
MSE  : 1.26
RMSE : 1.12
Precision: 0.86
Recall: 0.74
F1-Score: 0.79
NDCG Score : 0.77


##### Genre

In [53]:
test_data_euc_genre = pd.merge(test_data, df_pred_euc_genre, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_genre

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,4.393867
1,249,Push,3.5,4.298582
2,246,Arizona Dream,4.0,4.354871
3,414,Finding Nemo,4.5,3.514208
4,555,Batman Returns,3.0,3.938438
...,...,...,...,...
21471,5,Pinocchio,5.0,3.817661
21472,66,Beverly Hills Cop,3.0,4.746752
21473,356,"Bourne Supremacy, The",4.0,4.333660
21474,239,28 Days Later,5.0,4.515105


In [54]:
print("Euclidean Similarity genre \n")
euc_genere = calculate_errors(test_data_euc_genre)
euc_genere.append(get_precision_recall_f1(test_data_euc_genre))
euc_genere.append(ndcg_at_k(test_data_euc_genre))

Euclidean Similarity genre 

MAE  : 0.96
MSE  : 1.57
RMSE : 1.25
Precision: 0.83
Recall: 0.96
F1-Score: 0.89
NDCG Score : 0.83


##### Title

In [55]:
test_data_euc_title = pd.merge(test_data, df_pred_euc_title, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_title

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.282082
1,249,Push,3.5,2.899752
2,246,Arizona Dream,4.0,4.289055
3,414,Finding Nemo,4.5,3.317386
4,555,Batman Returns,3.0,3.363166
...,...,...,...,...
21471,5,Pinocchio,5.0,2.105824
21472,66,Beverly Hills Cop,3.0,4.063699
21473,356,"Bourne Supremacy, The",4.0,2.003235
21474,239,28 Days Later,5.0,4.051055


In [56]:
print("Euclidean Similarity title \n")
euc_title = calculate_errors(test_data_euc_title)
euc_title.append(get_precision_recall_f1(test_data_euc_title))
euc_title.append(ndcg_at_k(test_data_euc_title))

Euclidean Similarity title 

MAE  : 1.14
MSE  : 1.96
RMSE : 1.4
Precision: 0.87
Recall: 0.44
F1-Score: 0.59
NDCG Score : 0.64


##### Tag

In [57]:
test_data_euc_tag = pd.merge(test_data, df_pred_euc_tag, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_euc_tag

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,4.091558
1,249,Push,3.5,4.418665
2,246,Arizona Dream,4.0,5.348602
3,414,Finding Nemo,4.5,3.595484
4,555,Batman Returns,3.0,3.067748
...,...,...,...,...
21471,5,Pinocchio,5.0,0.894177
21472,66,Beverly Hills Cop,3.0,4.801323
21473,356,"Bourne Supremacy, The",4.0,5.512336
21474,239,28 Days Later,5.0,5.064312


In [58]:
print("Euclidean Similarity tag \n")
euc_tag = calculate_errors(test_data_euc_tag)
euc_tag.append(get_precision_recall_f1(test_data_euc_tag))
euc_tag.append(ndcg_at_k(test_data_euc_tag))

Euclidean Similarity tag 

MAE  : 1.11
MSE  : 1.94
RMSE : 1.39
Precision: 0.84
Recall: 0.75
F1-Score: 0.79
NDCG Score : 0.86


##Recomendation

In [59]:
user_id = 1

###Cosine Similarity

####All

In [60]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_all[df_pred_cos_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [61]:
get_recomendation_cos_all(user_id)

['Secret of Kells, The',
 'Blood: The Last Vampire',
 'The Green Inferno',
 'Counterfeiters, The (Die Fälscher)',
 'Aileen: Life and Death of a Serial Killer',
 'The Monkey King',
 'In the Mood For Love (Fa yeung nin wa)',
 'Proposition, The',
 'City of Lost Souls, The (Hyôryuu-gai)',
 'Darkest Hour, The']

####Story

In [62]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_story[df_pred_cos_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [63]:
get_recomendation_cos_story(user_id)

['Addiction, The',
 'World Trade Center',
 'White Chicks',
 'Rio 2',
 'Day Watch (Dnevnoy dozor)',
 'Fear City: A Family-Style Comedy (La cité de la peur)',
 'Apollo 18',
 'Blood: The Last Vampire',
 'Fireproof',
 'Maximum Overdrive']

####Genre

In [64]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_genre[df_pred_cos_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [65]:
get_recomendation_cos_genre(user_id)

["Pervert's Guide to Ideology, The",
 'Gigantic (A Tale of Two Johns)',
 'Weather Underground, The',
 'Food, Inc.',
 '42 Up',
 'Most Hated Family in America, The',
 'Kurt Cobain: Montage of Heck',
 'Going Clear: Scientology and the Prison of Belief',
 'Spellbound',
 'What Happened, Miss Simone?']

####Title

In [66]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_title[df_pred_cos_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [67]:
get_recomendation_cos_title(user_id)

['Nocturnal Animals',
 'Moonlight',
 nan,
 nan,
 'Steamboat Willie',
 'Battleship Potemkin',
 'Safety Last!',
 'Steamboat Bill, Jr.',
 'Alexander Nevsky (Aleksandr Nevskiy)',
 'Pygmalion']

####Tag

In [68]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_cos_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_cos_tag[df_pred_cos_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [69]:
get_recomendation_cos_tag(user_id)

['Getting Even with Dad',
 'Ciao, Professore! (Io speriamo che me la cavo)',
 'Shadowlands',
 'Outsourced',
 'Sleepless in Seattle',
 'Terminal Velocity',
 'Nightmare Before Christmas, The',
 'Welcome to the Dollhouse',
 'Immortal Beloved',
 'Heavenly Creatures']

###Euclidean Similarity

####All

In [70]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_all(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_all[df_pred_euc_all['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [71]:
get_recomendation_euc_all(user_id)

['Major League II',
 'Chasers',
 'Bob Roberts',
 'Sister Act 2: Back in the Habit',
 'Major League: Back to the Minors',
 'House Party 3',
 'Destiny Turns on the Radio',
 'Fled',
 'Schizopolis',
 'New Waterford Girl']

####Story

In [72]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_story(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_story[df_pred_euc_story['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [73]:
get_recomendation_euc_story(user_id)

['Self-criticism of a Bourgeois Dog',
 "Big Momma's House",
 'Rainmaker, The',
 'Oz the Great and Powerful',
 'Don Camillo in Moscow',
 'Man Who Knew Too Much, The',
 'Fled',
 'Dinotopia',
 "Fool's Gold",
 'RocknRolla']

####Genre

In [74]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_genre(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_genre[df_pred_euc_genre['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [75]:
get_recomendation_euc_genre(user_id)

['Fantastic Beasts and Where to Find Them',
 'Andalusian Dog, An (Chien andalou, Un)',
 'Meshes of the Afternoon',
 "Miss Peregrine's Home for Peculiar Children",
 'Sharknado',
 'Buck Rogers in the 25th Century',
 'Meteor',
 'Dawn of the Planet of the Apes',
 'Idaho Transfer',
 'Guyver: Dark Hero']

####Title

In [76]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_title(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_title[df_pred_euc_title['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [77]:
get_recomendation_euc_title(user_id)

['AVPR: Aliens vs. Predator - Requiem',
 "Happenstance (Battement d'ailes du papillon, Le)",
 'Dark Water (Honogurai mizu no soko kara)',
 "King Solomon's Mines",
 'Wet Hot American Summer',
 'Wicked City (Yôjû toshi)',
 'Beijing Bicycle (Shiqi sui de dan che)',
 'School Daze',
 'No Holds Barred',
 "Mr. Popper's Penguins"]

####Tag

In [78]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation_euc_tag(user_id):
    user_mov = new_ratings_refined[new_ratings_refined['userId'] == user_id]
    user_mov_pred = df_pred_euc_tag[df_pred_euc_tag['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [79]:
get_recomendation_euc_title(user_id)

['AVPR: Aliens vs. Predator - Requiem',
 "Happenstance (Battement d'ailes du papillon, Le)",
 'Dark Water (Honogurai mizu no soko kara)',
 "King Solomon's Mines",
 'Wet Hot American Summer',
 'Wicked City (Yôjû toshi)',
 'Beijing Bicycle (Shiqi sui de dan che)',
 'School Daze',
 'No Holds Barred',
 "Mr. Popper's Penguins"]

##Develop

### Pred_Rating data round

In [80]:
test_data_cos_all_round = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_round

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [81]:
test_data_cos_all_round['pred_rating'] = test_data_cos_all_round['pred_rating'].round(1)

In [82]:
print("Cosine_simility All_Round \n")
cos_all = calculate_errors(test_data_cos_all_round)
cos_all.append(get_precision_recall_f1(test_data_cos_all_round))
cos_all.append(ndcg_at_k(test_data_cos_all_round))

Cosine_simility All_Round 

MAE  : 0.92
MSE  : 1.28
RMSE : 1.13
Precision: 0.86
Recall: 0.63
F1-Score: 0.73
NDCG Score : 0.63


### Coverage Set

In [107]:
test_data_cos_all_Coverage = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_Coverage

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [108]:
threshold = 3
filtered_data = test_data_cos_all_Coverage[(test_data_cos_all_Coverage['rating'] > threshold) &
                                           (test_data_cos_all_Coverage['pred_rating'] > threshold)]
filtered_data

,userId,title,rating,pred_rating
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
6,246,Forrest Gump,3.5,3.272620
7,221,It's a Wonderful Life,5.0,3.130157
9,201,So I Married an Axe Murderer,5.0,3.929419
...,...,...,...,...
21467,450,North by Northwest,4.0,3.116578
21470,5,Pinocchio,5.0,4.420902
21471,5,Pinocchio,5.0,4.865209
21473,356,"Bourne Supremacy, The",4.0,3.672244


In [109]:
print("Cosine_simility All_filtered \n")
cos_all = calculate_errors(filtered_data)
cos_all.append(get_precision_recall_f1(filtered_data))
cos_all.append(ndcg_at_k(filtered_data))

Cosine_simility All_filtered 

MAE  : 0.75
MSE  : 0.83
RMSE : 0.91
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
NDCG Score : 0.69


###Round + Coverage Set

In [97]:
test_data_cos_all_Coverage_Round = pd.merge(test_data, df_pred_cos_all, on=['userId', 'title'], how='left') # userid와 title을 기준으로 합치기
test_data_cos_all_Coverage_Round

,userId,title,rating,pred_rating
0,313,Star Wars: Episode VI - Return of the Jedi,3.0,3.027375
1,249,Push,3.5,2.997994
2,246,Arizona Dream,4.0,3.538682
3,414,Finding Nemo,4.5,3.148872
4,555,Batman Returns,3.0,3.594758
...,...,...,...,...
21471,5,Pinocchio,5.0,4.865209
21472,66,Beverly Hills Cop,3.0,3.590364
21473,356,"Bourne Supremacy, The",4.0,3.672244
21474,239,28 Days Later,5.0,3.013930


In [104]:
threshold = 3
Round_filtered_data = test_data_cos_all_Coverage_Round[(test_data_cos_all_Coverage_Round['rating'] > threshold) &
                                           (test_data_cos_all_Coverage_Round['pred_rating'] > threshold)]

Round_filtered_data['pred_rating'] = Round_filtered_data['pred_rating'].round(1)

<ipython-input-104-e3602484a28c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Round_filtered_data['pred_rating'] = Round_filtered_data['pred_rating'].round(1)


In [106]:
print("Cosine_simility All_filtered \n")
cos_all = calculate_errors(Round_filtered_data)
cos_all.append(get_precision_recall_f1(Round_filtered_data))
cos_all.append(ndcg_at_k(Round_filtered_data))

Cosine_simility All_filtered 

MAE  : 0.75
MSE  : 0.83
RMSE : 0.91
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
NDCG Score : 0.69


## Supplement

### ALS + All_dot Hybrid

In [2]:
!pip install pyspark
!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
!apt-get install openjdk-8-jdk-headless -qq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   164  100   164    0     0   1670      0 --:--:-- --:--:-- --:--:--  1673
100   146  100   146    0     0    402      0 --:--:-- --:--:-- --:--:--   402
curl: (23) Failure writing output to destination


In [15]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [35]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.model_selection import train_test_split
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

spark_df_refined = spark.createDataFrame(new_ratings_refined)

# 사용자 ID와 아이템 ID(예: 영화 제목)를 숫자형으로 변환
user_indexer = StringIndexer(inputCol="userId", outputCol="user_id")
item_indexer = StringIndexer(inputCol="movieId", outputCol="item_id")
# Pipeline 생성
pipeline = Pipeline(stages=[user_indexer, item_indexer])
# 데이터 변환 실행
transformed_data = pipeline.fit(spark_df_refined).transform(spark_df_refined)
train_spark, test_spark = transformed_data.randomSplit([0.8, 0.2])


In [36]:
train_spark.show()

+------+-------+------+--------------------+-------+-------+
|userId|movieId|rating|               title|user_id|item_id|
+------+-------+------+--------------------+-------+-------+
|     1|      1|   4.0|           Toy Story|  111.0|   11.0|
|     1|      3|   4.0|    Grumpier Old Men|  111.0|  422.0|
|     1|      6|   4.0|                Heat|  111.0|  129.0|
|     1|     47|   5.0|Seven (a.k.a. Se7en)|  111.0|   15.0|
|     1|     50|   5.0| Usual Suspects, The|  111.0|   14.0|
|     1|     70|   3.0| From Dusk Till Dawn|  111.0|  390.0|
|     1|    151|   5.0|             Rob Roy|  111.0|  534.0|
|     1|    157|   5.0|      Canadian Bacon|  111.0| 1991.0|
|     1|    163|   5.0|           Desperado|  111.0|  280.0|
|     1|    223|   3.0|              Clerks|  111.0|  121.0|
|     1|    231|   5.0|Dumb & Dumber (Du...|  111.0|   63.0|
|     1|    260|   5.0|Star Wars: Episod...|  111.0|    5.0|
|     1|    296|   3.0|        Pulp Fiction|  111.0|    2.0|
|     1|    316|   3.0| 

In [37]:
# 훈련 데이터와 테스트 데이터로 분할
rec = ALS(maxIter=10,
         regParam=0.01,
         userCol='user_id',
         itemCol='item_id',
         ratingCol='rating',
         nonnegative=True,
         coldStartStrategy='drop')
# ALS모델 학습
rec_model = rec.fit(train_spark)

# transform을 이용해 예측
als_pred = rec_model.transform(test_spark)

In [39]:
als_pred.show()

+------+-------+------+--------------------+-------+-------+----------+
|userId|movieId|rating|               title|user_id|item_id|prediction|
+------+-------+------+--------------------+-------+-------+----------+
|   159|   3949|   0.5| Requiem for a Dream|  251.0|  148.0|  2.120237|
|    21|  66934|   0.5|Dr. Horrible's Si...|   53.0| 1088.0| 4.3002415|
|    65|   7458|   3.5|                Troy|  472.0|  471.0| 2.9154923|
|   282|   3752|   3.0|  Me, Myself & Irene|  108.0|  496.0| 4.1610622|
|   282|   3949|   4.5| Requiem for a Dream|  108.0|  148.0|  4.521784|
|   111|  84772|   3.0|                Paul|   28.0| 1645.0|  4.080062|
|   151|    135|   3.0|      Down Periscope|  336.0|  833.0| 3.6587844|
|   180|   1748|   3.5|           Dark City|  548.0|  463.0|  4.182232|
|   119|   7458|   4.5|                Troy|  122.0|  471.0| 3.7392383|
|   119| 114662|   5.0|     American Sniper|  122.0| 1580.0| 4.4588428|
|   201|   1748|   1.0|           Dark City|  230.0|  463.0| 4.6

In [40]:
# PySpark DataFrame을 Pandas DataFrame으로 변환
als_pred_pd = als_pred.toPandas()

selected_columns = ['userId','title', 'rating', 'prediction']
selected_als_pred_pd = als_pred_pd[selected_columns]

# Pandas의 merge 함수를 사용해 병합
hybrid_df = pd.merge(selected_als_pred_pd, df_pred_cos_all, left_on=['userId', 'title'], right_on=['userId', 'title'], suffixes=('_als', '_cos'))

# 가중치를 적용하여 하이브리드 평점 생성
hybrid_df['pred_rating'] = 0.8 * hybrid_df['prediction'] + 0.3 * hybrid_df['pred_rating']

# print(hybrid_df[['userId', 'hybrid_rating']])

In [50]:
print("hybrid_df All \n")
cos_all = calculate_errors(hybrid_df)
cos_all.append(get_precision_recall_f1(hybrid_df))
cos_all.append(ndcg_at_k(hybrid_df))

hybrid_df All 

MAE  : 0.74
MSE  : 0.98
RMSE : 0.99
Precision: 0.88
Recall: 0.88
F1-Score: 0.88
NDCG Score : 0.75


###SGD + All_dot Hybrid

In [92]:
selected_columns = ['userId','movieId', 'rating']
new_data = new_ratings_refined[selected_columns]

In [94]:
import numpy as np
import pandas as pd

class MatrixFactorization:
    def __init__(self, R, K, learning_rate, reg_param, epochs):
        """
        :param R: 사용자-아이템 평점 행렬 (데이터프레임)
        :param K: 잠재 요인의 차원
        :param learning_rate: 학습률
        :param reg_param: 정규화 파라미터
        :param epochs: 반복 횟수
        """
        self.R = R.values  # 데이터프레임을 numpy 배열로 변환
        self.num_users, self.num_items = self.R.shape
        self.K = K
        self.learning_rate = learning_rate
        self.reg_param = reg_param
        self.epochs = epochs

        # 잠재 요인 행렬 초기화
        self.P = np.random.normal(scale=1.0 / self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1.0 / self.K, size=(self.num_items, self.K))

    def train(self):
        for epoch in range(self.epochs):
            for i in range(self.num_users):
                for j in range(self.num_items):
                    if self.R[i][j] > 0:
                        eij = self.R[i][j] - np.dot(self.P[i, :], self.Q[j, :].T)
                        for k in range(self.K):
                            self.P[i][k] += self.learning_rate * (2 * eij * self.Q[j][k] - self.reg_param * self.P[i][k])
                            self.Q[j][k] += self.learning_rate * (2 * eij * self.P[i][k] - self.reg_param * self.Q[j][k])

            # 예측값을 특정 범위로 클리핑
            self.P = np.clip(self.P, 0, 5)
            self.Q = np.clip(self.Q, 0, 5)

            # 오차 계산
            error = self.calculate_error()
            print(f"Epoch {epoch + 1}/{self.epochs}, Error: {error}")

    def calculate_error(self):
        """
        모델의 현재 오차 계산
        """
        error = 0
        for i in range(self.num_users):
            for j in range(self.num_items):
                if self.R[i][j] > 0:
                    error += (self.R[i][j] - np.dot(self.P[i, :], self.Q[j, :].T))**2
                    for k in range(self.K):
                        error += (self.reg_param / 2) * (self.P[i][k]**2 + self.Q[j][k]**2)
        return error

    def predict(self):
        """
        학습된 모델을 사용하여 전체 사용자-아이템 평점 행렬 예측
        """
        return np.dot(self.P, self.Q.T)

# 잠재 요인의 차원, 학습률, 정규화 파라미터, 반복 횟수 등을 설정
K = 2
learning_rate = 0.01
reg_param = 0.02
epochs = 50

# 모델 생성 및 학습
mf = MatrixFactorization(new_data, K, learning_rate, reg_param, epochs)
mf.train()

# 예측 결과 출력
predicted_R = mf.predict()
predicted_R = np.clip(predicted_R, 0, 5)

<ipython-input-94-fcb0642e57a2>:31: RuntimeWarning: overflow encountered in double_scalars
  self.P[i][k] += self.learning_rate * (2 * eij * self.Q[j][k] - self.reg_param * self.P[i][k])
<ipython-input-94-fcb0642e57a2>:32: RuntimeWarning: overflow encountered in double_scalars
  self.Q[j][k] += self.learning_rate * (2 * eij * self.P[i][k] - self.reg_param * self.Q[j][k])


Epoch 1/50, Error: nan
Epoch 2/50, Error: nan
Epoch 3/50, Error: nan
Epoch 4/50, Error: nan
Epoch 5/50, Error: nan
Epoch 6/50, Error: nan
Epoch 7/50, Error: nan
Epoch 8/50, Error: nan
Epoch 9/50, Error: nan
Epoch 10/50, Error: nan
Epoch 11/50, Error: nan
Epoch 12/50, Error: nan
Epoch 13/50, Error: nan
Epoch 14/50, Error: nan
Epoch 15/50, Error: nan
Epoch 16/50, Error: nan
Epoch 17/50, Error: nan
Epoch 18/50, Error: nan
Epoch 19/50, Error: nan
Epoch 20/50, Error: nan
Epoch 21/50, Error: nan
Epoch 22/50, Error: nan
Epoch 23/50, Error: nan
Epoch 24/50, Error: nan
Epoch 25/50, Error: nan
Epoch 26/50, Error: nan
Epoch 27/50, Error: nan
Epoch 28/50, Error: nan
Epoch 29/50, Error: nan
Epoch 30/50, Error: nan
Epoch 31/50, Error: nan
Epoch 32/50, Error: nan
Epoch 33/50, Error: nan
Epoch 34/50, Error: nan
Epoch 35/50, Error: nan
Epoch 36/50, Error: nan
Epoch 37/50, Error: nan
Epoch 38/50, Error: nan
Epoch 39/50, Error: nan
Epoch 40/50, Error: nan
Epoch 41/50, Error: nan
Epoch 42/50, Error: nan
E

In [ ]:
predicted_R_sgd = predicted_R.toPandas()

# Merge ALS and Spark predictions
hybrid_df_sgd = pd.merge(predicted_R_sgd, df_pred_cos_all, on=['userId'], suffixes=('_sgd', '_spark'))

# Apply weights to create hybrid predictions
hybrid_df_sgd['hybrid_rating'] = 0.7 * hybrid_df_sgd['sgd_rating'] + 0.3 * hybrid_df_sgd['pred_rating']

# Display hybrid ratings
print(hybrid_df_sgd[['userId', 'hybrid_rating']])

In [ ]:
print("Cosine_simility All \n")
cos_all = calculate_errors(hybrid_df_sgd)
cos_all.append(get_precision_recall_f1(hybrid_df_sgd))
cos_all.append(ndcg_at_k(hybrid_df_sgd))